# 26. 나라면 이런 거 사겠다(2) - 대형주 짬뽕전략

[26강의](https://www.youtube.com/watch?v=vaeLRY5u3MU) 를 파이썬으로 구현해보겠습니다. [11강의 파이썬](https://github.com/tomowind/hkkang-youtube/blob/master/notebooks/11-value-quality-momentum-korea.ipynb) 과 거의 흡사하네요. 

## 설치

Pandas 가 있어야 합니다. 설치가 안되어있으시면 다음 셀을 실행시키세요.

In [1]:
import sys
!{sys.executable} -m pip install pandas xlrd openpyxl

## 다운로드

[네이버 카페 퀀트킹(QuantKing)](https://cafe.naver.com/quantking)에서 최신 엑셀 파일을 다운받으세요.

저는 **QuantKing(2019.01.25).zip** 파일을 받았습니다. 받으면 확장자 **xlsb** 파일이 보입니다. 엑셀로 여셔서 **QuantKing.xlsx** 파일로 다시 저장해주세요.

## 파일 로딩

파일을 열어서 조사해 보겠습니다. Pandas의 read_excel() 이란 함수를 이용하면 열 수 있네요.

In [2]:
import pandas as pd

In [3]:
filepath = 'QuantKing.xlsx'
df = pd.read_excel(filepath, sheet_name='퀀트데이타', skiprows=2).drop('Unnamed: 0', axis=1)
df.head()

,코드 번호,회사명,업종 (대),업종 (소),코스피코스닥,주가 (원),시가총액 (억),상장주식수 (만주),자사주 (만주),자사주 비중 (%),...,18년 2Q.2,18년 3Q.2,18년 4Q(E).2,19년 1Q(E).2,18년4Q 매출액,18년4Q 영업이익,18년4Q 순이익,18년4Q 매출액.1,18년4Q 영업이익.1,18년4Q 순이익.1
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,44750,2671478,596978.0,0,0.0,...,109815,129674,100441.00,79168.0,590000.00,108000.00,NaN,631283.0,133354.0,100441.0
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,74600,543090,72800.0,4400,6.0,...,43299,46936,33962.90,16797.0,99380.81,44300.75,33962.90,101549.0,50660.0,40819.0
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,128500,274564,21367.0,939,4.4,...,7006,2692,-1297.74,7829.0,256695.34,5011.27,-1297.74,261108.0,7611.0,6727.0
3,A068270,셀트리온,제약 및 바이오,바이오,코스피,208500,266772,12795.0,85,0.7,...,840,562,544.00,521.0,NaN,NaN,NaN,2512.0,792.0,544.0
4,A207940,삼성바이오로직스,제약 및 바이오,바이오,코스피,403000,266645,6617.0,0,0.0,...,-96,-301,4362.00,28.0,NaN,NaN,NaN,1393.0,149.0,4362.0


강의에서 사용하는 컬럼만 잘라서 보관하도록 하겠습니다.

In [4]:
df = df[['회사명',
         '시가총액\n(억)', # 시가총액
         '1개월\n등락률\n(%)', '3개월\n등락율\n(%)', '6개월\n등락율\n(%)', '9개월\n등락율\n(%)', '1년\n등락율\n(%)', # 모멘텀
         '과거\nGP/A\n(%)', '주가\n변동성', '이익의\n변동성', '자산성장율\n(연간기준)', 'F스코어\n점수 (9점만점)', # 우량주
         '발표\nPBR', '과거\nPER', '과거\nPCR', '과거\nPSR', '시가\n배당률\n(%)' # 저평가
        ]].set_index('회사명').sort_values(by='시가총액\n(억)', ascending=False).iloc[:200]
df.head()

,시가총액 (억),1개월 등락률 (%),3개월 등락율 (%),6개월 등락율 (%),9개월 등락율 (%),1년 등락율 (%),과거 GP/A (%),주가 변동성,이익의 변동성,자산성장율 (연간기준),F스코어 점수 (9점만점),발표 PBR,과거 PER,과거 PCR,과거 PSR,시가 배당률 (%)
회사명,,,,,,,,,,,,,,,,
삼성전자,2671478,17.0,5.7,-1.8,-15.6,-11.9,35.3,0.01720,0.204499,15.1,7,1.14,5.62,4.03,1.07,1.9
SK하이닉스,543090,21.1,9.4,-10.3,-14.4,-1.2,43.7,0.02344,0.351748,41.0,7,1.16,3.53,2.71,1.37,1.3
현대차,274564,8.4,19.5,2.4,-18.7,-15.7,8.8,0.01940,0.373443,-0.4,6,0.40,10.28,5.42,0.29,3.1
셀트리온,266772,-6.5,-5.9,-24.6,-23.2,-29.3,19.4,0.03486,0.298549,14.5,6,10.53,78.90,60.02,26.31,0.0
삼성바이오로직스,266645,8.6,1.3,6.6,-20.4,-3.6,2.4,0.03688,-0.583107,-4.6,6,7.13,-249.20,472.92,50.91,0.0


## 대형주는 왜?

- 2016-17년 대형주 만세, 소형주 퍽퍽
    - 자괴감 + 유체이탈 = 계량투자 포기 가능!
- 따라서 대형주 포함 추천, 계량으로 추가 수익
    - 단, 기대수익은 주가지수 + 한자리 정도

## 모멘텀 순위

In [5]:
df['3-1 모멘텀'] = df['3개월\n등락율\n(%)'] - df['1개월\n등락률\n(%)']
df['6-1 모멘텀'] = df['6개월\n등락율\n(%)'] - df['1개월\n등락률\n(%)']
df['9-1 모멘텀'] = df['9개월\n등락율\n(%)'] - df['1개월\n등락률\n(%)']
df['12-1 모멘텀'] = df['1년\n등락율\n(%)'] - df['1개월\n등락률\n(%)']

df['3순위'] = df['3-1 모멘텀'].rank(ascending=False)
df['6순위'] = df['6-1 모멘텀'].rank(ascending=False)
df['9순위'] = df['9-1 모멘텀'].rank(ascending=False)
df['12순위'] = df['12-1 모멘텀'].rank(ascending=False)

df['모멘텀 통합순위'] = df[['3순위', '6순위', '9순위', '12순위']].mean(axis=1)

df.sort_values(by='모멘텀 통합순위').head()

,시가총액 (억),1개월 등락률 (%),3개월 등락율 (%),6개월 등락율 (%),9개월 등락율 (%),1년 등락율 (%),과거 GP/A (%),주가 변동성,이익의 변동성,자산성장율 (연간기준),...,시가 배당률 (%),3-1 모멘텀,6-1 모멘텀,9-1 모멘텀,12-1 모멘텀,3순위,6순위,9순위,12순위,모멘텀 통합순위
회사명,,,,,,,,,,,,,,,,,,,,,
아난티,22597,31.0,215.9,251.5,264.1,404.6,7.8,0.06270,NaN,8.5,...,0.0,184.9,220.5,233.1,373.6,1.0,1.0,1.0,1.0,1.000
한진칼,16656,-11.5,51.3,52.6,13.1,23.5,10.6,0.03156,-12.028931,34.6,...,0.4,62.8,64.1,24.6,35.0,2.0,2.0,13.5,14.0,7.875
휠라코리아,28419,-8.8,14.3,53.0,83.8,158.9,51.5,0.03064,1.689849,-9.0,...,0.1,23.1,61.8,92.6,167.7,27.0,3.0,2.0,2.0,8.500
대우조선해양,37040,-0.6,24.1,32.1,37.1,59.6,4.5,0.03168,-8.787771,-24.0,...,0.0,24.7,32.7,37.7,60.2,22.0,8.0,5.0,7.0,10.500
대림산업,36192,1.0,38.5,31.3,25.6,28.1,11.1,0.02150,0.769942,8.2,...,1.0,37.5,30.3,24.6,27.1,7.0,11.0,13.5,17.0,12.125


## 우량주 순위

In [6]:
'과거\nGP/A\n(%)', '주가\n변동성', '이익의\n변동성', '자산성장율\n(연간기준)', 'F스코어\n점수 (9점만점)', # 우량주

df['GP/A 순위'] = df['과거\nGP/A\n(%)'].rank(ascending=False)
df['주가 변동성 순위'] = df['주가\n변동성'].rank(ascending=True)
df['이익 변동성 순위'] = df['이익의\n변동성'].rank(ascending=True)
df['자산 성장률 순위'] = df['자산성장율\n(연간기준)'].rank(ascending=True)
df['F스코어 순위'] = df['F스코어\n점수 (9점만점)'].rank(ascending=False)

df['우량주 통합순위'] = df['GP/A 순위'] * 0.15 + \
                    df['주가 변동성 순위'] * 0.15 + \
                    df['이익 변동성 순위'] * 0.15 + \
                    df['자산 성장률 순위'] * 0.15 + \
                    df['F스코어 순위'] * 0.4

df.sort_values(by='우량주 통합순위').head()

,시가총액 (억),1개월 등락률 (%),3개월 등락율 (%),6개월 등락율 (%),9개월 등락율 (%),1년 등락율 (%),과거 GP/A (%),주가 변동성,이익의 변동성,자산성장율 (연간기준),...,6순위,9순위,12순위,모멘텀 통합순위,GP/A 순위,주가 변동성 순위,이익 변동성 순위,자산 성장률 순위,F스코어 순위,우량주 통합순위
회사명,,,,,,,,,,,,,,,,,,,,,
롯데쇼핑,55163,-8.7,0.0,-5.3,-23.8,-18.6,28.1,0.02105,-9.460926,-33.3,...,56.0,104.0,66.0,79.00,52.0,72.0,3.0,3.0,4.5,21.300
LG유플러스,72914,-1.8,4.0,5.7,35.2,11.3,93.8,0.01988,0.093048,-0.4,...,44.0,6.0,21.0,44.25,5.0,58.0,31.0,39.0,39.0,35.550
이노션,13640,6.7,22.0,26.3,2.4,-3.9,27.1,0.02087,0.250431,-2.9,...,25.5,66.0,71.5,53.75,56.0,69.0,55.0,26.0,15.0,36.900
제일기획,26114,1.1,9.1,16.1,18.2,11.5,50.9,0.02009,0.344545,4.0,...,32.0,22.0,23.0,42.50,23.0,62.0,78.0,87.5,15.0,43.575
SK텔레콤,217206,1.1,0.0,6.3,18.8,0.6,44.9,0.01440,0.257584,6.8,...,50.0,21.0,37.0,64.50,28.0,9.5,57.0,115.5,39.0,47.100


## 가치 순위

In [7]:
'발표\nPBR', '과거\nPER', '과거\nPCR', '과거\nPSR', '시가\n배당률\n(%)'

df = df[df['발표\nPBR'] != 0]
df = df[df['과거\nPER'] != 0]
df = df[df['과거\nPCR'] != 0]
df = df[df['과거\nPSR'] != 0]

df['1/PBR'] = 1 / df['발표\nPBR']
df['1/PER'] = 1 / df['과거\nPER']
df['1/PCR'] = 1 / df['과거\nPCR']
df['1/PSR'] = 1 / df['과거\nPSR']

df['순위 PBR'] = df['1/PBR'].rank(ascending=False)
df['순위 PER'] = df['1/PER'].rank(ascending=False)
df['순위 PCR'] = df['1/PCR'].rank(ascending=False)
df['순위 PSR'] = df['1/PSR'].rank(ascending=False)
df['순위 시가 배당률'] = df['1/PSR'].rank(ascending=False)
df['통합 가치 지표'] = df[['순위 PBR', '순위 PER', '순위 PCR', '순위 PSR', '순위 시가 배당률']].mean(axis=1)

df.sort_values(by='통합 가치 지표').head()

,시가총액 (억),1개월 등락률 (%),3개월 등락율 (%),6개월 등락율 (%),9개월 등락율 (%),1년 등락율 (%),과거 GP/A (%),주가 변동성,이익의 변동성,자산성장율 (연간기준),...,1/PBR,1/PER,1/PCR,1/PSR,순위 PBR,순위 PER,순위 PCR,순위 PSR,순위 시가 배당률,통합 가치 지표
회사명,,,,,,,,,,,,,,,,,,,,,
한화,25374,6.3,23.8,6.1,-15.5,-27.9,3.0,0.01925,1.051433,3.4,...,1.694915,0.192308,0.833333,20.000000,31.0,16.0,1.5,1.0,1.0,10.1
HDC,10305,-0.6,3.3,-31.8,-50.0,-47.0,31.3,0.03242,1.425535,13.0,...,1.851852,0.900901,0.325733,5.263158,22.0,2.0,22.0,14.5,14.5,15.0
효성,12643,11.7,22.4,33.3,-19.5,-22.3,32.8,0.04212,2.460374,2.9,...,1.369863,2.325581,0.423729,10.000000,50.0,1.0,16.0,4.0,4.0,15.0
GS,48873,4.2,11.2,0.4,-12.9,-23.4,20.3,0.01577,0.329242,8.2,...,1.694915,0.193050,0.311526,3.703704,31.0,15.0,24.0,25.0,25.0,24.0
CJ,35158,0.0,18.9,-9.0,-20.1,-32.7,36.0,0.01875,1.197224,8.6,...,1.204819,0.127389,0.431034,8.333333,67.0,39.0,14.5,6.5,6.5,26.7


## 통합순위

AQR 의 논문처럼 <span style="color:red">**저평가 지표 * 0.4 + 모멘텀 지표 * 0.4 + 우량주 지표 * 0.2**</span>

In [8]:
df['통합순위'] = df['모멘텀 통합순위'] * 0.4 + df['통합 가치 지표'] * 0.4 + df['우량주 통합순위'] * 0.2
df.sort_values(by='통합순위')[['시가총액\n(억)', '모멘텀 통합순위', '통합 가치 지표', '우량주 통합순위', '통합순위']].head(n=20)

,시가총액 (억),모멘텀 통합순위,통합 가치 지표,우량주 통합순위,통합순위
회사명,,,,,
대림산업,36192,12.125,31.3,72.750,31.920
기아차,144917,43.125,34.1,62.700,43.430
대한항공,33006,45.625,37.2,81.450,49.420
LG유플러스,72914,44.250,70.2,35.550,52.890
롯데쇼핑,55163,79.000,46.3,21.300,54.380
KT,76375,62.500,40.4,71.250,55.410
현대해상,33525,63.125,35.0,84.475,56.145
메리츠화재,25351,41.375,51.5,108.525,58.855
GS건설,37653,55.500,53.8,86.325,60.985
